In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
tfb = tfp.bijectors
from gp_models import CustomKernel,GP_model, GPR_model, GPR_analytical

In [7]:
# Code to import S&P500 call option data and match them with daily S&P500 
# and VIX levels from Yahoo Finance

paths = ['vix_data/SPX_Index/SPX_30DAY_IMPVOL_allMNY.csv','vix_data/SPX_Index/SPX_60DAY_IMPVOL_allMNY.csv'
        ,'vix_data/SPX_Index/SPX_3MTH_IMPVOL_allMNY.csv','vix_data/SPX_Index/SPX_6MTH_IMPVOL_allMNY.csv'
        ,'vix_data/SPX_Index/SPX_12MTH_IMPVOL_allMNY.csv']

df = pd.read_csv('vix_data/SPX_Index/SPX_30DAY_IMPVOL_allMNY.csv')

maturities = [30,60,90,180,360]
m = 0
cols = ['date', 'XDAY_IMPVOL_80.MNY_DF', 'XDAY_IMPVOL_90.0.MNY_DF',
       'XDAY_IMPVOL_95.0.MNY_DF', 'XDAY_IMPVOL_97.5.MNY_DF',
       'XDAY_IMPVOL_100.0.MNY_DF', 'XDAY_IMPVOL_102.5.MNY_DF',
       'XDAY_IMPVOL_105.0.MNY_DF', 'XDAY_IMPVOL_110.0.MNY_DF',
       'XDAY_IMPVOL_120.MNY_DF', 'Date', 'tau', 'mid_spx', 'mid_vix']
df0 = pd.DataFrame(columns=cols)
vix = yf.Ticker("^VIX")
sp500 = yf.Ticker("^GSPC")

start_date = datetime.datetime(2017,1,1)

for p in paths:
    df = pd.read_csv(p)
    df.columns = cols[:-4]
    df['Date'] =df.date.apply(datetime.datetime.strptime,args = ('%Y-%m-%d',))
    
    df = df[df['Date']>start_date]
    df['tau'] = maturities[m]
    end_of_data = df['Date'].iloc[-1]
 
    n_days = int((datetime.datetime.now()-start_date).days)
    period = str(n_days)+'d'
    df_spx = sp500.history(period=period).iloc[:,:-2].reset_index()
    df_spx['mid_spx'] = 1/2*(df_spx.High+df_spx.Low)

    df_vix = vix.history(period=period).iloc[:,:-2].reset_index()
    df_vix['mid_vix'] = 1/2*(df_vix.High+df_vix.Low)

    df_spx = df_spx[df_spx['Date']<end_of_data+datetime.timedelta(days=1)]
    df_vix = df_vix[df_vix['Date']<end_of_data+datetime.timedelta(days=1)]

    spx = df_spx[['Date','mid_spx']]
    vixx = df_vix[['Date','mid_vix']]

    df_all = pd.merge(df,spx,on='Date')
    df_all = pd.merge(df_all,vixx,on='Date')

    df0 = pd.concat([df0,df_all],axis=0)
    m+=1
    break
    
df0.shape

(1058, 14)

In [8]:
#Construct the training data input points matrix
last_day = datetime.datetime(2017,7,1)
include_swap = True

# Use this to select just one value of time to maturity
maturity = 30
df_all = df0[df0['tau']==maturity]
df_train = df_all[df_all['Date']<last_day]

# Use this to keep multiple values of tau and the tau column
# df_train = df0[df0['Date']<last_day]

# Use this to add a timestamp to the datapoints
# df_train['t'] = np.arange(1,len(df_train)+1)

# Drop the date columns as they are not inputs to the model
df_train = df_train.drop(['Date','date'],axis=1)

# Sample one day out of two to reduce data size
df_train = df_train.iloc[::2,:]
n_days = df_train.shape[0]

# Match SPX and VIX data to option prices per day (9 quoted prices per day)
columns_to_keep = ['mid_spx','mid_vix'] # '['tau','mid_spx','mid_vix']' or ['mid_spx','mid_vix','t']
moneyness = np.array([80,90,95,97.5,100,102.5,105,110,120])

indices = df_train[columns_to_keep].values
indices = np.repeat(indices,len(moneyness),axis=0)

mny = np.tile(moneyness,n_days)
# Observations
y = df_train.loc[:,:'XDAY_IMPVOL_120.MNY_DF'].values.flatten()
# Inputs
X = np.concatenate([np.reshape(mny,[-1,1]),indices],axis=1)

if not include_swap:
    X = X[:,:-1]
    X_test = X_test[:,:-1]

# Convert to tensor
X = tf.constant(X,dtype=tf.float64)

In [11]:
# Define all hyperparameters as trainable TensorFlow Variables
# Note that the length of the length_scales list must be equal to the input dimension

process_variance = 32
noise_variance = 0.18
length_scales =[12.7,502]
length_scales = [12.7,502,16.08]
length_scales = [12.7,65,270]
lower_bounds = [0.01,10]
upper_bounds = [3,50]
trend_coef = [[1,1]]
if include_swap:
    lower_bounds.append(0.001)
    upper_bounds.append(2.)
    length_scales.append(16.08)
    #length_scales.append(20)
    
    trend_coef[0].append(1)
# Mean function hyperparameters
b0 = tf.Variable(16.59,dtype=tf.float64, name = 'b0') 
b1 = tf.Variable(1,dtype=tf.float64, name= 'b1')
b2 = tf.Variable(1,dtype=tf.float64, name= 'b2')
b = tf.Variable(trend_coef,dtype=tf.float64, name= 'b')
# Kernel hyperparameters
sp = tfp.util.TransformedVariable(process_variance, tfb.Exp(), dtype=tf.float64, name='sigma_process')
sn2 = tfp.util.TransformedVariable(noise_variance, tfb.Exp(), dtype=tf.float64, name='sigma_noise2')
scales = tf.Variable(length_scales,dtype=tf.float64)#,constraint = lambda x: tf.clip_by_value(x,lower_bounds,upper_bounds))#,dtype=tf.float64, name = 'length_scales')

# Define custom kernel
kernel_family = 'M52'
param ={'amp' : sp , 'scales': scales,'family' : kernel_family}
my_kernel = CustomKernel(1,parameters = param)

#Define mean function
#def mean_f(x): return tf.squeeze(b0 + tf.linalg.matmul(x, b, transpose_b=True))
def mean_f(x): return b0 
#def mean_f(x): return 0

In [ ]:
#Define and fit the model
GP = GP_model(my_kernel, index_points=X,mean_fn=mean_f, beta=(b0,b),observation_noise_variance=sn2,jitter=1e-06, name='GP')
GP.fit(y,n_iters=500,verbose=True,optimizer = tf.optimizers.Adam(0.01))
print('\n## Trained Hyperparameters ##\n')
print(sp)
print(scales)
print(sn2)
print(b0)
#GP.hyperparam

In [13]:
# Choose a day to test the predictions on
# 'future' indicates how many days after the last training day the test set is chosen
future = 4
n_days = df_train.shape[0]*2
test_date = df_all.iloc[n_days+future-1,0]
df_test = df_all.iloc[n_days+future-1,1:].drop(['Date'])
y_test = df_test[:9].values

# Choose between timestamped or unmarked data
X_test = np.array([moneyness,np.ones(9)*df_test['mid_spx'],np.ones(9)*df_test['mid_vix']]).T

t_test = X[-1,-1]+future
X_test = np.array([moneyness,np.ones(9)*df_test['mid_spx'],np.ones(9)*df_test['mid_vix'],
                 np.ones(9)*t_test]).T
test_date

'2017-07-10'

In [ ]:
#Define a GP Regression model with learned parameters, scalar noise variance
GPR = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=sn2,predictive_noise_variance=None,mean_fn=mean_f,jitter=1e-6)


In [17]:
# GPR model with vector of noise variances (the code to find the weights is further down)
weights = tf.constant(np.linspace(9,0.1,int(n_days/2)),dtype=tf.float64)
nugget = weights*sn2
new_sn2 = tf.repeat(nugget,9)

GPR = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=new_sn2,predictive_noise_variance=sn2,mean_fn=mean_f,jitter=1e-6)


In [ ]:
# Plot predictions, confidence bounds and market implied volatility for test set

from sklearn.metrics import mean_squared_error as mse

plt.scatter(moneyness,GPR.price(),marker='x',c='r',label='GP estimate')
plt.plot(moneyness,GPR.bounds_price(0.95)[0],'--',c='black',label = '95% confidence bounds')
plt.plot(moneyness,GPR.bounds_price(0.95)[1],'--',c='black')
plt.plot(moneyness,y_test,label='Market Implied Volatility')
plt.legend()
plt.xlabel('Moneyness')
plt.ylabel('Implied volatility')
rmse = np.sqrt(mse(y_test,GPR.price()))
plt.title('RMSE = '+str(round(rmse,4)))
fig_path = '/Users/danielmontagna/Documents/ETHZ/PH-MA3/Thesis/Report/thesis/Figures/VIX/'
fig_name = 'pred_5days_cumul_m52.pdf'
fig_path += fig_name
#plt.savefig(fig_path)

In [ ]:
# Test predictions for model with different maturities as inputs

error_array = np.zeros([50,5])
dates = np.unique(df0[df0['Date']>last_day].Date)
for i in range(50):
    the_day = last_day + datetime.timedelta(days=i+1)
    the_day = dates[i]
    df_test = df0[df0['Date']==the_day]

    m = 0
    errs =[]
    for mat in maturities:
        y_test = df_test.iloc[m,1:10]
        X_test = np.array([moneyness,mat*np.ones(9),np.ones(9)*df_test['mid_spx'].mean()
                           ,np.ones(9)*df_test['mid_vix'].mean()]).T
        GPR = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=sn2
                    ,predictive_noise_variance=None,mean_fn=mean_f,jitter=1e-6) 
#         plt.plot(moneyness,y_test,'--',label=r'$\tau =$ '+str(mat)+ ' days')
#         plt.scatter(moneyness,GPR.mean(),marker='d')
        errs.append(mse(y_test,GPR.mean()))
        m+=1
    error_array[i,:] = np.array(errs)
# plt.legend()
# plt.xlabel('Moneyness')
# plt.ylabel('Implied Volatility')
# #plt.show()
# fig_path = '/Users/danielmontagna/Documents/ETHZ/PH-MA3/Thesis/Report/thesis/Figures/VIX/'
# fig_name = 'pred_5days_5taus.pdf'
# fig_path += fig_name
#plt.savefig(fig_path)
print(np.mean(error_array,axis=0))
error_array

In [ ]:
# Plot predictions for different maturities for one day on the same graph
from sklearn.metrics import mean_squared_error as mse
future = 4
n_days = df_train.shape[0]*2

test_date = df0[df0['tau']==mat].iloc[40+future,0]
df_test = df0[df0['tau']==mat].iloc[40+future,1:].drop(['Date'])
for mat in maturities:
    y_test = df_test[:9].values
    X_test = np.array([moneyness,mat*np.ones(9),np.ones(9)*df_test['mid_spx'],np.ones(9)*df_test['mid_vix']]).T
    gpr = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=sn2,predictive_noise_variance=None,mean_fn=mean_f,jitter=1e-6)

    plt.plot(moneyness,gpr.mean(),'d')
    plt.plot(moneyness,y_test,'--')
    erri = np.sqrt(mse(y_test,gpr.mean()))
print(np.mean(erri))
plt.show()

In [ ]:
#Grid search to find w_max and w_min

from sklearn.metrics import mean_squared_error as mse
width = 10
cross_val = np.zeros([width,width])
maxs = np.linspace(1,10,width)
mins = np.linspace(0.1,1,width)
j = 0
future = 5
for wmin in mins:
    k= 0
    for wmax in maxs:
        errs = []
        weights = tf.constant(np.linspace(wmax,wmin,int(n_days/4)),dtype=tf.float64)
        nugget = weights*sn2
        new_sn2 = tf.repeat(nugget,9)
        #for i in range(10):
        half = 62

        df_test = df_all.iloc[half:half+future,1:].drop(['Date'],axis=1)
        y_test = df_test.iloc[:,:9].values.flatten()
        indices = df_test.iloc[:,-2:].values
        indices = np.repeat(indices,len(moneyness),axis=0)

        mny = np.tile(moneyness,future)

        X_test = np.concatenate([np.reshape(mny,[-1,1]),indices],axis=1)

        end = (31)*9
            
        GPR = GPR_model(my_kernel,X_test,X[:end,:],y[:end],observation_noise_variance=new_sn2,predictive_noise_variance=sn2,mean_fn=mean_f,jitter=1e-6)

        err=mse(y_test,GPR.price())
            #errs.append(err)
        #cross_val[j,k]=np.mean(errs)
        cross_val[j,k]=err
        
        k+=1
    j+=1
# print('AVG RMSE = ',np.mean(errs))
# print('MAX = ',np.max(errs))
# print('MIN = ',np.min(errs))
# plt.plot(ws,cross_val,'d',label = 'RMSE')
# plt.xlabel('$w_{min}$',fontsize='14')
# plt.ylabel('AVG RMSE (14 days)')
# plt.legend()
# fig_path = '/Users/danielmontagna/Documents/ETHZ/PH-MA3/Thesis/Report/thesis/Figures/VIX/'
# fig_path+= 'grid_search_wmax.pdf'
# plt.tight_layout()
#plt.savefig(fig_path)

cross_val

In [ ]:
weights = tf.constant(np.linspace(10,0.1,int(n_days/2)),dtype=tf.float64)
nugget = weights*sn2
new_sn2 = tf.repeat(nugget,9)
new_sn2.shape

In [ ]:
# Plot results of the grid search

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
YY,XX = np.meshgrid(maxs,mins)
ZZ = cross_val
ax.plot_surface(YY,XX,ZZ,cmap = cm.coolwarm)
ax.set_xlabel(r'$w_{max}$',fontsize='14')
ax.set_ylabel(r'$w_{min}$',fontsize='14')
ax.set_zlabel('MSE',rotation='90')

plt.savefig('/Users/danielmontagna/Documents/ETHZ/PH-MA3/Thesis/Report/thesis/Figures/VIX/w_surface.pdf')

In [ ]:
# Compute average RMSE over a period following the training data ( e.g. D_m +7 or D_m +50) 

weights = tf.constant(np.linspace(9,0.1,int(n_days/2)),dtype=tf.float64)

nugget = weights*sn2
new_sn2 = tf.repeat(nugget,9)
errs=[]
for i in np.arange(1,7):
    future = i
    df_test = df_all.iloc[n_days+future-1,1:].drop(['Date'])

    y_test = df_test[:9].values
    #X_test = np.array([moneyness,np.ones(9)*df_test['mid_spx'],np.ones(9)*df_test['mid_vix']]).T
    t_test = X[-1,-1]+future
    X_test = np.array([moneyness,np.ones(9)*df_test['mid_spx'],np.ones(9)*df_test['mid_vix'],
                 np.ones(9)*t_test]).T
    #Define a GPR model with scalar noise (sn2) or vector valued noise (new_sn2)
    GPR = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=new_sn2,predictive_noise_variance=None,mean_fn=mean_f,jitter=1e-6)
    #GPR = GPR_model(my_kernel,X_test,X,y,observation_noise_variance=sn2,predictive_noise_variance=sn2,mean_fn=mean_f,jitter=1e-6)

    rmse = np.sqrt(mse(y_test,GPR.price()))
    errs.append(rmse)
print('AVG RMSE = ',np.mean(errs))
print('MAX = ',np.max(errs))
print('MIN = ',np.min(errs))
print('MEDIAN = ',np.median(errs))

In [ ]:
# Plot historical S&P500 and VIX levels

df_spx = sp500.history(period=period,interval='1d').iloc[:,:-2].reset_index()
df_spx['mid_spx'] = 1/2*(df_spx.High+df_spx.Low)

df_vix = vix.history(period=period,interval='1d').iloc[:,:-2].reset_index()
df_vix['mid_vix'] = 1/2*(df_vix.High+df_vix.Low)

last_day = datetime.datetime(2017,10,1)

df_spx = df_spx[df_spx['Date']<last_day+datetime.timedelta(days=1)]
df_vix = df_vix[df_vix['Date']<last_day+datetime.timedelta(days=1)]

df_spx = df_spx[df_spx['Date']>start_date-datetime.timedelta(days=1)]
df_vix = df_vix[df_vix['Date']>start_date-datetime.timedelta(days=1)]


# df_spx = df_spx.iloc[:-275,:]
# df_vix = df_vix.iloc[:-275,:]


fig, ax1 = plt.subplots()
ax1.set_xlabel('Date')
ax1.set_ylabel('S&P 500')
days = np.arange(df_spx.shape[0])

ax1.plot(np.arange(df_spx.shape[0]),df_spx.mid_spx,c='black',label = 'S&P 500')
months = ['01/2017','02/2017','03/2017','04/2017','05/2017','06/2017','07/2017','08/2017','09/2017']
plt.xticks(ticks = np.arange(0,len(days),22),labels = months,rotation = 70)
ax2 = ax1.twinx() 
ax2.set_ylabel('VIX')
ax2.plot(days,df_vix.mid_vix,c='tab:blue',label='VIX')
ax2.vlines(133,9,16,linestyles='dashed',colors='red')
ax2.legend()
ax1.legend()
fig.tight_layout()
plt.savefig('index_history_2017.pdf')